In [3]:
import numpy as np
import pandas as pd
import string
#import xgboost as xgb
import io
import nltk
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('stopwords')
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

from textblob import TextBlob
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matiascano/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
sia = SentimentIntensityAnalyzer()
def return_sia_compound_values(text):
    return sia.polarity_scores(text)['compound']

In [5]:
def remove_stopword(text):
    new_text = []
    for e in text:
        if e not in stopwords and e.isalpha():
            new_text.append(e)
    text = new_text
    return " ".join(new_text)

def stemm(text):
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

def contains_punctuation(text):
    punctuation = set(string.punctuation)
    for character in text:
        if character in punctuation:
            return True
    return False

def amount_of_punctuation(text):
    punctuation = set(string.punctuation)
    amount = 0
    for character in text:
        if character in punctuation: amount += 1
    return amount

def get_adjectives(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("JJ")])

def get_nouns(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("NN")])

def get_verbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("VB")])

def get_adverbs(text):
    blob = TextBlob(text)
    return len([word for (word,tag) in blob.tags if tag.startswith("RB")])

In [27]:
tweets = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
tweets['keyword'] = tweets.keyword.str.replace('%20',' ')

In [7]:
tweets.drop_duplicates(subset = 'text', keep = False, inplace = True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7434 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7434 non-null   int64 
 1   keyword   7378 non-null   object
 2   location  4982 non-null   object
 3   text      7434 non-null   object
 4   target    7434 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 348.5+ KB


In [8]:
test['text'].duplicated().value_counts()

False    3243
True       20
Name: text, dtype: int64

### Feature Engineering

In [9]:
tweets_metrics = tweets[['id','text','target']]
tweets_metrics['text_without_stopwords'] = tweets_metrics['text'].str.split()
tweets_metrics['text_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(remove_stopword)

tweets_metrics['length'] = tweets_metrics['text'].apply(lambda x: len(x))
tweets_metrics['avg_word_length'] = tweets_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
tweets_metrics['amount_of_words'] = tweets_metrics['text'].str.split().transform(lambda x: len(x))
unique_words_by_tweet = tweets_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
tweets_metrics['amount_of_unique_words'] = unique_words_by_tweet
tweets_metrics['sentiment'] = tweets_metrics['text'].apply(lambda x: return_sia_compound_values(x))
tweets_metrics['stopwords_count'] = tweets_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
tweets_metrics['punctuation_count'] = tweets_metrics['text'].apply(lambda x: amount_of_punctuation(x))
mentions = tweets_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()
tweets_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
hashtags = tweets_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
tweets_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
tweets_metrics['longest_word_length_without_stopwords'] = tweets_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
tweets_metrics['stopword_word_ratio'] = tweets_metrics['stopwords_count'] / tweets_metrics['amount_of_words']

tweets_metrics['adjectives_count'] = tweets_metrics['text'].apply(get_adjectives)
tweets_metrics['nouns_count'] = tweets_metrics['text'].apply(get_nouns)
tweets_metrics['verbs_count'] = tweets_metrics['text'].apply(get_verbs)
tweets_metrics['adverbs_count'] = tweets_metrics['text'].apply(get_adverbs)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
tweets_metrics.to_csv('train_features.csv', index=False)

In [11]:
test_metrics = test[['id','text']]
test_metrics['text_without_stopwords'] = test_metrics['text'].str.split()
test_metrics['text_without_stopwords'] = test_metrics['text_without_stopwords'].apply(remove_stopword)

test_metrics['length'] = test_metrics['text'].apply(lambda x: len(x))
test_metrics['avg_word_length'] = test_metrics['text'].str.split().apply(lambda x: [len(y) for y in x]).transform(lambda x: np.mean(x))
test_metrics['amount_of_words'] = test_metrics['text'].str.split().transform(lambda x: len(x))
unique_words_by_tweet = test_metrics['text'].transform(lambda x: x.split()).transform(lambda x: pd.Series(x).unique()).transform(lambda x: len(x))
test_metrics['amount_of_unique_words'] = unique_words_by_tweet
test_metrics['sentiment'] = test_metrics['text'].apply(lambda x: return_sia_compound_values(x))
test_metrics['stopwords_count'] = test_metrics['text'].apply(lambda x: len([word for word in str(x).lower().split() if word in stopwords]))
test_metrics['punctuation_count'] = test_metrics['text'].apply(lambda x: amount_of_punctuation(x))
mentions = test_metrics['text'].str.findall(r'@.\S*?(?=\s|[:]|$)').to_frame()
test_metrics['mentions_count'] = mentions['text'].apply(lambda x: len(x))
hashtags = test_metrics['text'].str.findall(r'#[^?\s].*?(?=\s|$)')
test_metrics['hashtags_count'] = hashtags.apply(lambda x: len(x))
test_metrics['longest_word_length_without_stopwords'] = test_metrics['text_without_stopwords'].apply(lambda x: ([len(word) for word in str(x).lower().split() if not word.startswith('http')])).apply(lambda x: max(x) if len(x) > 0 else 0)
test_metrics['stopword_word_ratio'] = test_metrics['stopwords_count'] / test_metrics['amount_of_words']

test_metrics['adjectives_count'] = test_metrics['text'].apply(get_adjectives)
test_metrics['nouns_count'] = test_metrics['text'].apply(get_nouns)
test_metrics['verbs_count'] = test_metrics['text'].apply(get_verbs)
test_metrics['adverbs_count'] = test_metrics['text'].apply(get_adverbs)

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
test.to_csv('test_features.csv', index=False)

### Word2Vec

#### Keywords

In [28]:
tweets['keyword'] = tweets['keyword'].fillna('NULL')
test['keyword'] = test['keyword'].fillna('NULL')

In [29]:
keywords = tweets[['keyword', 'id']]
keywords.head()

,keyword,id
0,NULL,1
1,NULL,4
2,NULL,5
3,NULL,6
4,NULL,7


In [30]:
keyword_tokens = keywords.keyword.unique().tolist()

In [31]:
keyword_tokens[0]

'NULL'

In [32]:
len(keyword_tokens)

222

In [33]:
from gensim.models import Word2Vec

# Parameters
# sg ({0, 1}, optional) - Training algorithm: 1 for skip-gram; otherwise CBOW.

keyword_vectors = Word2Vec([keyword_tokens], min_count=1, size= 100, workers=3, window =3, sg=1)

In [34]:
# Sanity check
keyword_vectors['ablaze']

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-2.9782532e-03, -4.6778829e-03,  2.6636261e-03,  1.3892590e-03,
       -4.1829501e-04, -2.2293918e-03, -4.5149596e-03,  3.5410111e-03,
       -1.2479150e-03,  1.1896417e-03,  4.9190470e-03, -2.1441414e-03,
        2.3400637e-03,  2.6803168e-03, -1.8570827e-03, -2.0209479e-03,
       -4.4037956e-03,  1.8780368e-03, -3.9049496e-03, -4.4517661e-03,
       -6.0586346e-04,  4.4179936e-03,  1.9494499e-03, -1.3178665e-03,
       -2.5463670e-03,  3.7509988e-03, -9.1646833e-04, -1.0098880e-03,
        2.1787037e-03, -1.8912348e-03, -3.4733759e-03, -3.4088921e-04,
       -3.4694674e-03, -1.8341526e-03, -4.8559005e-04, -2.9973371e-03,
       -2.6874964e-03, -9.6715416e-04, -3.2191214e-03,  4.3508359e-03,
        6.1339342e-05,  3.0415037e-03,  2.6784365e-03,  1.7985675e-03,
        1.7651119e-03, -3.9943969e-03, -2.8382847e-03,  1.4784534e-03,
       -4.0552719e-03,  4.1793748e-03,  2.5931003e-03, -2.2637420e-03,
        3.3299834e-03,  8.0845342e-04, -6.7411043e-04,  1.7605064e-03,
      

In [35]:
to_vector_matrix = {}

for k in keyword_tokens:
    to_vector_matrix[k] = keyword_vectors[k]

/Users/matiascano/.pyenv/versions/3.7.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [36]:
keyword_w2v = pd.DataFrame.from_dict(to_vector_matrix).T.reset_index()
keyword_w2v

,index,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,NULL,-0.002684,-0.001899,-0.000798,-0.001893,0.003133,0.002236,-0.001299,-0.003281,-0.004414,...,-0.002605,-0.004927,0.001684,0.004702,-0.004160,0.003781,-0.003558,0.003921,0.000945,-0.002725
1,ablaze,-0.002978,-0.004678,0.002664,0.001389,-0.000418,-0.002229,-0.004515,0.003541,-0.001248,...,0.002117,0.003154,0.004220,0.004947,-0.000484,0.003842,0.003322,0.002055,0.003155,0.002922
2,accident,-0.003384,0.002203,0.000089,0.003893,0.001827,0.001415,0.002242,-0.003304,0.000639,...,0.002065,0.000828,0.000448,-0.004822,-0.004947,0.003784,0.003708,-0.000240,0.004170,0.004662
3,aftershock,0.001398,0.003535,-0.002235,-0.003819,-0.004369,-0.004255,-0.001596,0.000770,0.003828,...,0.002328,-0.002230,0.000262,0.001067,0.002255,0.004911,-0.002622,0.001655,-0.000919,-0.000489
4,airplane accident,-0.002153,-0.003184,-0.000135,-0.000386,-0.003797,-0.001352,0.002524,-0.001643,0.001755,...,-0.000326,-0.001173,0.004366,0.001570,0.001540,0.001214,0.002249,0.004110,0.002640,-0.000939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,wounded,0.001796,-0.003270,0.004549,0.004226,0.002533,0.000455,0.004593,0.002722,0.003202,...,-0.000337,-0.004437,-0.000958,0.004536,0.001090,0.000493,-0.001115,0.000474,0.000006,-0.003310
218,wounds,-0.001814,0.003487,0.003526,-0.001831,-0.000307,0.002191,0.004440,0.002222,-0.002227,...,0.004556,-0.001981,0.000711,-0.003419,-0.004136,0.004411,-0.001755,-0.002368,-0.003373,-0.004597
219,wreck,0.001275,0.001559,-0.004270,0.003340,-0.001561,0.002126,-0.003265,-0.002016,-0.000342,...,0.002935,0.003843,-0.001886,0.003956,0.001262,-0.003812,0.001750,-0.000785,-0.000671,-0.004001
220,wreckage,0.004161,0.000882,0.003419,0.000611,0.000960,-0.003009,-0.003214,0.003773,-0.003924,...,-0.001317,-0.000323,0.001569,-0.000505,-0.004607,0.003270,0.003372,0.002703,-0.004637,0.004220


In [37]:
aux = []
aux.append('keyword')
for i in range (0, 100):
    name = 'v' + str(i)
    aux.append(name)
len(aux)

101

In [38]:
keyword_w2v.columns = aux

In [39]:
keyword_w2v.head()

,keyword,v0,v1,v2,v3,v4,v5,v6,v7,v8,...,v90,v91,v92,v93,v94,v95,v96,v97,v98,v99
0,NULL,-0.002684,-0.001899,-0.000798,-0.001893,0.003133,0.002236,-0.001299,-0.003281,-0.004414,...,-0.002605,-0.004927,0.001684,0.004702,-0.004160,0.003781,-0.003558,0.003921,0.000945,-0.002725
1,ablaze,-0.002978,-0.004678,0.002664,0.001389,-0.000418,-0.002229,-0.004515,0.003541,-0.001248,...,0.002117,0.003154,0.004220,0.004947,-0.000484,0.003842,0.003322,0.002055,0.003155,0.002922
2,accident,-0.003384,0.002203,0.000089,0.003893,0.001827,0.001415,0.002242,-0.003304,0.000639,...,0.002065,0.000828,0.000448,-0.004822,-0.004947,0.003784,0.003708,-0.000240,0.004170,0.004662
3,aftershock,0.001398,0.003535,-0.002235,-0.003819,-0.004369,-0.004255,-0.001596,0.000770,0.003828,...,0.002328,-0.002230,0.000262,0.001067,0.002255,0.004911,-0.002622,0.001655,-0.000919,-0.000489
4,airplane accident,-0.002153,-0.003184,-0.000135,-0.000386,-0.003797,-0.001352,0.002524,-0.001643,0.001755,...,-0.000326,-0.001173,0.004366,0.001570,0.001540,0.001214,0.002249,0.004110,0.002640,-0.000939


In [40]:
keyword_w2v.to_csv('keyword_features.csv', index=False)

#### Locations (proximamente)